In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
! which python

/home/local_arnab/miniconda3/envs/fact/bin/python


In [3]:
import sys
sys.path.append('..')

import torch
torch.__version__, torch.version.cuda

('1.13.1+cu117', '11.7')

In [4]:
from src import models, data, lens, functional
from src.utils import experiment_utils, env_utils

import os
import logging
from src.utils import logging_utils
logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

In [5]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# model_name = "EleutherAI/pythia-2.8b-deduped"

# models_dir = env_utils.determine_models_dir()

# model_path = os.path.join(models_dir, model_name)

# model = AutoModelForCausalLM.from_pretrained(model_path)
# tokenizer = AutoTokenizer.from_pretrained(model_path)

In [7]:
device = "cuda:0"
# model_name = "mamba-3b"
model_name = "pythia-3b"

mt = models.load_model(model_name, device=device, fp16=False)
# mt = models.load_model("gptj", device=device, fp16=True)

2024-02-15 12:58:22 src.models DEBUG    EleutherAI/pythia-2.8b-deduped not downloadable, will look for weights in /home/local_arnab/Codes/relations/notebooks/../../saved_model_weights
2024-02-15 12:58:22 src.models INFO     loading /home/local_arnab/Codes/relations/notebooks/../../saved_model_weights/EleutherAI/pythia-2.8b-deduped (device=cuda:0, fp16=False)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


2024-02-15 12:58:25 src.models INFO     dtype: torch.float32, device: cuda:0, memory: 10.47 GB


In [20]:
# import baukit

# mt.model.config.fused_add_norm = False

# for block_path in models.determine_layer_paths(mt):
#     block = baukit.get_module(mt.model, block_path)
#     setattr(block, "fused_add_norm", False)

In [21]:
mt.model.config

GPTNeoXConfig {
  "_name_or_path": "/home/local_arnab/Codes/relations/notebooks/../../saved_model_weights/EleutherAI/pythia-2.8b-deduped",
  "architectures": [
    "GPTNeoXForCausalLM"
  ],
  "attention_bias": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.1,
  "eos_token_id": 0,
  "hidden_act": "gelu",
  "hidden_dropout": 0.0,
  "hidden_size": 2560,
  "initializer_range": 0.02,
  "intermediate_size": 10240,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neox",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "rope_scaling": null,
  "rotary_emb_base": 10000,
  "rotary_pct": 0.25,
  "tie_word_embeddings": false,
  "transformers_version": "4.37.2",
  "use_cache": true,
  "use_parallel_residual": true,
  "vocab_size": 50304
}

In [22]:
from src.functional import predict_next_token

predict_next_token(
    mt = mt, 
    prompt = mt.tokenizer.eos_token + " The capital of {} is".format("France"),
    # prompt = mt.tokenizer.eos_token + " The superlative of {} is".format("good"),
)

[[PredictedToken(token=' Paris', prob=0.24318307638168335),
  PredictedToken(token=' a', prob=0.09811916202306747),
  PredictedToken(token=' the', prob=0.08357568830251694),
  PredictedToken(token=' one', prob=0.04043750464916229),
  PredictedToken(token=' located', prob=0.01906432770192623)]]

In [23]:
dataset = data.load_dataset()

# relation_names = [r.name for r in dataset.relations]
# relation_options = Menu(choices = relation_names, value = relation_names)
# show(relation_options) # !caution: tested in a juputer-notebook. baukit visualizations are not supported in vscode.

2024-02-15 12:45:23 src.data DEBUG    no paths provided, using default data dir: /home/local_arnab/Codes/relations/notebooks/../data
2024-02-15 12:45:23 src.data DEBUG    /home/local_arnab/Codes/relations/notebooks/../data is directory, globbing for json files...
2024-02-15 12:45:23 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/characteristic_gender.json
2024-02-15 12:45:23 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/degree_gender.json
2024-02-15 12:45:23 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/name_birthplace.json
2024-02-15 12:45:23 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/name_gender.json
2024-02-15 12:45:23 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/name_religion.json
2024-02-15 12:45:23 src.data DEBUG    found relation 

In [24]:
relation_name = "country capital city"
relation = dataset.filter(relation_names=[relation_name])[0]
print(f"{relation.name} -- {len(relation.samples)} samples")
print("------------------------------------------------------")

experiment_utils.set_seed(12345) # set seed to a constant value for sampling consistency
train, test = relation.split(5)
print("\n".join([sample.__str__() for sample in train.samples]))

2024-02-15 12:45:25 src.data DEBUG    filtering to only relations: ['country capital city']
country capital city -- 24 samples
------------------------------------------------------
2024-02-15 12:45:25 src.utils.experiment_utils INFO     setting all seeds to 12345
China -> Beijing
Japan -> Tokyo
Italy -> Rome
Brazil -> Bras\u00edlia
Turkey -> Ankara


In [28]:
################### hparams ###################
layer = 12
beta = 5
###############################################

In [30]:
from src.operators import JacobianIclMeanEstimator
experiment_utils.set_seed(12345)

estimator = JacobianIclMeanEstimator(
    mt = mt, 
    h_layer = layer,
    beta = beta
)
operator = estimator(
    relation.set(
        samples=train.samples, 
    )
)

2024-02-15 12:46:05 src.utils.experiment_utils INFO     setting all seeds to 12345
2024-02-15 12:46:05 src.operators WARNING  relation has > 1 prompt_templates, will use first (The capital city of {} is)
2024-02-15 12:46:05 src.operators DEBUG    estimating J for prompt:
<|endoftext|> The capital city of Japan is Tokyo
The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of Turkey is Ankara
The capital city of China is
2024-02-15 12:46:05 src.operators DEBUG    note that subject=China, h_index=43
2024-02-15 12:46:08 src.operators DEBUG    estimating J for prompt:
<|endoftext|> The capital city of China is Beijing
The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of Turkey is Ankara
The capital city of Japan is
2024-02-15 12:46:08 src.operators DEBUG    note that subject=Japan, h_index=43
2024-02-15 12:46:11 src.operators DEBUG    estimating J for prompt:
<|endoftext|> The capital city of China is Bei

In [31]:
# from typing import Optional

# def add_npz_extension(path: str):
#     return path if path.endswith(".npz") else path + ".npz"

# def load_o1_approxes(path: str, sample_subjects: Optional[list[str]] = None):
#     approxes = []
#     to_load = sample_subjects if sample_subjects is not None else os.listdir(path)
#     for cached_file in to_load:
#         file_path = add_npz_extension(os.path.join(path, cached_file))
#         approx = functional.load_cached_linear_operator(file_path = file_path)
#         approxes.append(approx)
#     return approxes

In [22]:
# ##################################################
# n_train_samples = 5
# ##################################################

In [32]:
# import os
# import random

# root_path = "../results/cached_o1_approxes"
# path = os.path.join(
#     root_path,
#     mt.name,
#     relation_name.lower().replace(" ", "_"),
#     str(layer),
# )

# all_cached_files = list(os.listdir(path))

# train_subj_files = random.sample(all_cached_files, n_train_samples)

# train_approxes = load_o1_approxes(
#     path = path, 
#     sample_subjects = train_subj_files
# )

# train_samples = [
#     data.RelationSample.from_dict(approx.metadata["sample"]) 
#     for approx in train_approxes
# ]

# train_relation = relation.set(samples=train_samples)

# test_relation = relation.set(
#     samples = list(set(relation.samples) - set(train_relation.samples))
# )

In [33]:
# weight = torch.stack([approx.weight for approx in train_approxes]).mean(dim=0)
# bias = torch.stack([approx.bias for approx in train_approxes]).mean(dim=0)

# prompt_template = relation.prompt_templates[0]

# prompt_template_icl = functional.make_prompt(
#     mt = mt,
#     prompt_template=prompt_template,
#     subject="{}",
#     examples = train_samples
# )

# print(prompt_template_icl)

In [34]:
# from src.operators import LinearRelationOperator
# operator = LinearRelationOperator(
#     mt = mt,
#     weight = weight,
#     bias = bias,
#     h_layer = train_approxes[0].h_layer,
#     z_layer = train_approxes[0].z_layer,
#     beta = 5,
#     prompt_template = prompt_template_icl,
#     metadata = {
#         "Jh": [
#             (approx.weight @ approx.h).detach().cpu().numpy()
#             for approx in train_approxes
#         ],
#         "|w|": [approx.weight.norm().item() for approx in train_approxes],
#         "|b|": [approx.bias.norm().item() for approx in train_approxes],
#     },
# )

In [35]:
from src.lens import logit_lens
from src import models

# logit_lens(mt = mt, h = operator.metadata["Jh"][0].to(models.determine_device(mt)) + operator.bias)

# Checking $faithfulness$

In [37]:
test = functional.filter_relation_samples_based_on_provided_fewshots(
    mt=mt, test_relation=test, prompt_template=operator.prompt_template, batch_size=4
)

2024-02-15 12:46:54 src.functional DEBUG    filtering for knowns using prompt "<|endoftext|> The capital city of China is Beijing
The capital city of Japan is Tokyo
The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of Turkey is Ankara
The capital city of {} is"


2024-02-15 12:46:55 src.functional DEBUG    sample.subject='South Korea', sample.object='Seoul', predicted=' Seoul' (p=0.973), known=(✓)
2024-02-15 12:46:55 src.functional DEBUG    sample.subject='Colombia', sample.object='Bogot\\u00e1', predicted=' Bog' (p=0.985), known=(✓)
2024-02-15 12:46:55 src.functional DEBUG    sample.subject='Saudi Arabia', sample.object='Riyadh', predicted=' R' (p=0.915), known=(✓)
2024-02-15 12:46:55 src.functional DEBUG    sample.subject='France', sample.object='Paris', predicted=' Paris' (p=0.985), known=(✓)
2024-02-15 12:46:55 src.functional DEBUG    sample.subject='Mexico', sample.object='Mexico City', predicted=' Mexico' (p=0.962), known=(✓)
2024-02-15 12:46:55 src.functional DEBUG    sample.subject='Pakistan', sample.object='Islamabad', predicted=' Islam' (p=0.683), known=(✓)
2024-02-15 12:46:55 src.functional DEBUG    sample.subject='Argentina', sample.object='Buenos Aires', predicted=' Buenos' (p=0.970), known=(✓)
2024-02-15 12:46:55 src.functional DE

In [38]:
sample = [s for s in test.samples if s.subject == "France"][0]
print(sample)
operator(subject = sample.subject).predictions

France -> Paris
2024-02-15 12:46:58 src.operators DEBUG    computing h from prompt "<|endoftext|> The capital city of China is Beijing
The capital city of Japan is Tokyo
The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of Turkey is Ankara
The capital city of France is"


[PredictedToken(token=' Paris', prob=0.9591746926307678),
 PredictedToken(token=' France', prob=0.028499744832515717),
 PredictedToken(token='Paris', prob=0.008190473541617393),
 PredictedToken(token=' French', prob=0.0023832363076508045),
 PredictedToken(token='France', prob=0.001406631083227694)]

In [39]:
hs_and_zs = functional.compute_hs_and_zs(
    mt = mt,
    prompt_template = operator.prompt_template,
    subjects = [sample.subject],
    h_layer= operator.h_layer,
)
h = hs_and_zs.h_by_subj[sample.subject]

## Approximating LM computation $F$ as an affine transformation

### $$ F(\mathbf{s}, c_r) \approx \beta \, W_r \mathbf{s} + b_r $$

In [40]:
z = 5 * (operator.weight @ h) + operator.bias

lens.logit_lens(
    mt = mt,
    h = z,
    get_proba = True
)

([(' Paris', 0.959),
  (' France', 0.028),
  ('Paris', 0.008),
  (' French', 0.002),
  ('France', 0.001),
  ('French', 0.0),
  (' Franco', 0.0),
  (' franc', 0.0),
  (' Rome', 0.0),
  (' Franc', 0.0)],
 {})

In [41]:
correct = 0
wrong = 0
for sample in test.samples:
    predictions = operator(subject = sample.subject).predictions
    known_flag = functional.is_nontrivial_prefix(
        prediction=predictions[0].token, target=sample.object
    )
    print(f"{sample.subject=}, {sample.object=}, ", end="")
    print(f'predicted="{functional.format_whitespace(predictions[0].token)}", (p={predictions[0].prob}), known=({functional.get_tick_marker(known_flag)})')
    
    correct += known_flag
    wrong += not known_flag
    
faithfulness = correct/(correct + wrong)

print("------------------------------------------------------------")
print(f"Faithfulness (@1) = {faithfulness}")
print("------------------------------------------------------------")

2024-02-15 12:47:14 src.operators DEBUG    computing h from prompt "<|endoftext|> The capital city of China is Beijing
The capital city of Japan is Tokyo
The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of Turkey is Ankara
The capital city of Argentina is"


sample.subject='Argentina', sample.object='Buenos Aires', predicted=" Buenos", (p=0.959989607334137), known=(✓)
2024-02-15 12:47:14 src.operators DEBUG    computing h from prompt "<|endoftext|> The capital city of China is Beijing
The capital city of Japan is Tokyo
The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of Turkey is Ankara
The capital city of Australia is"
sample.subject='Australia', sample.object='Canberra', predicted=" Sydney", (p=0.4635578691959381), known=(✗)
2024-02-15 12:47:14 src.operators DEBUG    computing h from prompt "<|endoftext|> The capital city of China is Beijing
The capital city of Japan is Tokyo
The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of Turkey is Ankara
The capital city of Canada is"
sample.subject='Canada', sample.object='Ottawa', predicted=" Ottawa", (p=0.35514500737190247), known=(✓)
2024-02-15 12:47:14 src.operators DEBUG    computing h from prompt "<|e

# $causality$

In [44]:
################### hparams ###################
rank = 100
###############################################

In [45]:
experiment_utils.set_seed(12345) # set seed to a constant value for sampling consistency
test_targets = functional.random_edit_targets(test.samples)

2024-02-15 12:47:36 src.utils.experiment_utils INFO     setting all seeds to 12345


## setup

In [46]:
source = test.samples[0]
target = test_targets[source]

f"Changing the mapping ({source}) to ({source.subject} -> {target.object})"

'Changing the mapping (Argentina -> Buenos Aires) to (Argentina -> Riyadh)'

### Calculate $\Delta \mathbf{s}$ such that $\mathbf{s} + \Delta \mathbf{s} \approx \mathbf{s}'$

<p align="center">
    <img align="center" src="../causality_crop" style="width:80%;"/>
</p>

Under the relation $r =\, $*plays the instrument*, and given the subject $s =\, $*Miles Davis*, the model will predict $o =\, $*trumpet* **(a)**; and given the subject $s' =\, $*Cat Stevens*, the model will now predict $o' =\, $*guiter* **(b)**. 

If the computation from $\mathbf{s}$ to $\mathbf{o}$ is well-approximated by $operator$ parameterized by $W_r$ and $b_r$ **(c)**, then $\Delta{\mathbf{s}}$ **(d)** should tell us the direction of change from $\mathbf{s}$ to $\mathbf{s}'$. Thus, $\tilde{\mathbf{s}}=\mathbf{s}+\Delta\mathbf{s}$ would be an approximation of $\mathbf{s}'$ and patching $\tilde{\mathbf{s}}$ in place of $\mathbf{s}$ should change the prediction to $o'$ = *guitar* 

In [47]:
def get_delta_s(
    operator, 
    source_subject, 
    target_subject,
    rank = 100,
    fix_latent_norm = None, # if set, will fix the norms of z_source and z_target
):
    w_p_inv = functional.low_rank_pinv(
        matrix = operator.weight,
        rank=rank,
    )
    hs_and_zs = functional.compute_hs_and_zs(
        mt = mt,
        prompt_template = operator.prompt_template,
        subjects = [source_subject, target_subject],
        h_layer= operator.h_layer,
        z_layer=-1,
    )

    z_source = hs_and_zs.z_by_subj[source_subject]
    z_target = hs_and_zs.z_by_subj[target_subject]
    
    z_source *= fix_latent_norm / z_source.norm() if fix_latent_norm is not None else 1.0
    z_target *= z_source.norm() / z_target.norm() if fix_latent_norm is not None else 1.0

    delta_s = w_p_inv @  (z_target.squeeze() - z_source.squeeze())

    return delta_s, hs_and_zs

delta_s, hs_and_zs = get_delta_s(
    operator = operator,
    source_subject = source.subject,
    target_subject = target.subject,
    rank = rank
)

In [48]:
import baukit

def get_intervention(h, int_layer, subj_idx):
    def edit_output(output, layer):
        if(layer != int_layer):
            return output
        functional.untuple(output)[:, subj_idx] = h 
        return output
    return edit_output

prompt = operator.prompt_template.format(source.subject)

h_index, inputs = functional.find_subject_token_index(
    mt=mt,
    prompt=prompt,
    subject=source.subject,
)

h_layer, z_layer = models.determine_layer_paths(model = mt, layers = [layer, -1])

with baukit.TraceDict(
    mt.model, layers = [h_layer, z_layer],
    edit_output=get_intervention(
#         h = hs_and_zs.h_by_subj[source.subject],         # let the computation proceed as usual
        h = hs_and_zs.h_by_subj[source.subject] + delta_s, # replace s with s + delta_s
        int_layer = h_layer, 
        subj_idx = h_index
    )
) as traces:
    outputs = mt(
        input_ids = inputs.input_ids,
        attention_mask = inputs.attention_mask,
    )

logits = outputs.logits[0][-1] if hasattr(outputs, "logits") else outputs[0][-1]
lens.interpret_logits(
    mt = mt, 
    logits = logits, 
    get_proba=True
)

[(' R', 0.896),
 (' Jed', 0.05),
 (' Me', 0.007),
 (' Saudi', 0.005),
 ('\n', 0.004),
 (' K', 0.004),
 (' D', 0.003),
 (' \\', 0.003),
 (' the', 0.002),
 (' Dh', 0.001)]

## Measuring causality

In [49]:
from src.editors import LowRankPInvEditor

svd = torch.svd(operator.weight.float())
editor = LowRankPInvEditor(
    lre=operator,
    rank=rank,
    svd=svd,
)

In [50]:
# precomputing latents to speed things up
hs_and_zs = functional.compute_hs_and_zs(
    mt = mt,
    prompt_template = operator.prompt_template,
    subjects = [sample.subject for sample in test.samples],
    h_layer= operator.h_layer,
    z_layer=-1,
    batch_size = 2
)

success = 0
fails = 0

for sample in test.samples:
    target = test_targets.get(sample)
    assert target is not None
    edit_result = editor(
        subject = sample.subject,
        target = target.subject
    )
    
    success_flag = functional.is_nontrivial_prefix(
        prediction=edit_result.predicted_tokens[0].token, target=target.object
    )
    
    print(f"Mapping {sample.subject} -> {target.object} | edit result={edit_result.predicted_tokens[0]} | success=({functional.get_tick_marker(success_flag)})")
    
    success += success_flag
    fails += not success_flag
    
causality = success / (success + fails)

print("------------------------------------------------------------")
print(f"Causality (@1) = {causality}")
print("------------------------------------------------------------")

2024-02-15 12:47:41 src.editors DEBUG    computing low-rank pinv (rel=<|endoftext|> The capital city of China is Beijing
The capital city of Japan is Tokyo
The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of Turkey is Ankara
The capital city of {} is)


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Mapping Argentina -> Riyadh | edit result=' R' (p=0.904) | success=(✓)
Mapping Australia -> Buenos Aires | edit result=' Buenos' (p=0.894) | success=(✓)
Mapping Canada -> Abuja | edit result=' Abu' (p=0.946) | success=(✓)


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Mapping Chile -> Lima | edit result=' L' (p=0.988) | success=(✓)
Mapping Colombia -> Berlin | edit result=' Berlin' (p=0.941) | success=(✓)
Mapping Egypt -> Mexico City | edit result=' Mexico' (p=0.955) | success=(✓)


...

Mapping Peru -> Caracas | edit result=' Car' (p=0.928) | success=(✓)
Mapping Russia -> Cairo | edit result=' Cairo' (p=0.881) | success=(✓)
Mapping Saudi Arabia -> Caracas | edit result=' R' (p=0.909) | success=(✗)


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Mapping South Korea -> Cairo | edit result=' Cairo' (p=0.864) | success=(✓)
Mapping Spain -> Islamabad | edit result=' Islam' (p=0.530) | success=(✓)
Mapping United States -> Ottawa | edit result=' Ottawa' (p=0.693) | success=(✓)


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Mapping Venezuela -> Madrid | edit result=' Madrid' (p=0.545) | success=(✓)
------------------------------------------------------------
Causality (@1) = 0.9473684210526315
------------------------------------------------------------


In [52]:
# from src.functional import save_linear_operator

# save_linear_operator(
#     approx = operator,
#     file_name = "lre_capital",
#     path = "cached"
# )

In [53]:
# operator_loaded = functional.load_cached_linear_operator(mt = mt, file_path = "cached/lre_capital.npz")

## Test

In [18]:
import numpy as np

beta_upper_limit = 12
np.linspace(0, beta_upper_limit, 25)

array([ 0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ,
        5.5,  6. ,  6.5,  7. ,  7.5,  8. ,  8.5,  9. ,  9.5, 10. , 10.5,
       11. , 11.5, 12. ])

In [20]:
layers = None

isinstance(layers, list) and len(layers) > 0

False